In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import os
from langchain.chains import LLMChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import pandas as pd
import numpy as np
import json

In [2]:
def str2json(json_string):
        # Split string into lines 
    lines = json_string.split('\n')

    # Initialize empty dictionary
    data = {}

    # Iterate through lines 
    for line in lines:
        # Split line into key-value pair
        try:
            if len(line)>2:
                parts = line.split(': ')
        
                # Extract key and value
                key = parts[0].strip('"').strip('{')
                value = parts[1].strip('}')
        
                # Add to dictionary
                data[key] = value
        except:
            continue

    return data

In [3]:
response_schemas = [
    ResponseSchema(name="aspect_term", description="aspect term"),
    ResponseSchema(name="sedntiment", description="sentiment, can be positive, neutral or negative")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [4]:
template = """This the review from Amazon customer. The tile of the review is: {title}. The content if the review is: {review}. 
You are expert in NLP and aspect based sentiment analysis. Please do aspect based sentential analysis with the review and the title in the \ 
following aspects:\
# Effectiveness
# Price
# Taste
# Side effects
# Quality control
# Varied results
# Ingredients. \

Please give out your reply in the json format, use the aspect term as key, and class(Neutral, positive, or negative) as value: 
aspect term: class(Neutral, positive, or negative.).
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
\n{aspect_term}\n{sentiment}
"""

In [5]:
prompt = PromptTemplate(
    template=template,
    input_variables=["title", "review"],
    partial_variables={"aspect_term": format_instructions, "sentiment":format_instructions}
)

In [6]:
model = OpenAI(temperature=0)

In [9]:
df = pd.read_csv('./data/ABSA.csv')
df_data = df.loc[:, ['title', 'content']]
df_data = df_data.iloc[:50,:]

df_data['aspect_sentiment'] = np.nan

for idx, row in df_data.iterrows():
    title_ = row['title']
    review_ = row['content']
    _input = prompt.format_prompt(title=title_, review=review_)
    output = model(_input.to_string())
    
    df_data.loc[idx, 'aspect_sentiment'] = output   

In [10]:
df_data.head()

,title,content,aspect_sentiment
0,UPDATED REVIEW - 5 months in,I have been taking these gummies (along with t...,"\n{""Effectiveness"": ""Positive"",\n""Price"": ""Neu..."
1,"This helped me slow down hair loss, but when I...","They are a little expensive, but they seem to ...",\nEffectiveness: Positive\nPrice: Neutral\nTas...
2,"The hype is real, I stand corrected",For so long I never bought these because I did...,"\n{""Effectiveness"": ""Positive"",\n""Price"": ""Neu..."
3,Excelente,Mi comentario positivo no es por que me hayan ...,\nEffectiveness: Positive\nPrice: Negative\nTa...
4,Do it!!,I have tried this product a few times over the...,"\n""Effectiveness"": ""Positive"",\n""Price"": ""Neut..."


In [95]:
#df_data.to_csv('absa.csv')

In [11]:
title_ = "The hype is real, I stand corrected"
review_ = "For so long I never bought these because I didn’t want to support the kardashians financially since they promoted it often, however now years later, I tried it and I love it. A family friend who went to cosmetology school, and hair school, recommended these to me after I told her that ever since I cut my hair super short in college I feel like my hair doesn’t grow pst a certain length anymore. I started taking these and my lashes are fuller and my brows are fuller! Best of all my hair is literally noticeably longer than it ever has been other than when I was a little girl. I didn’t want to believe it but it works. I will note that I have a sensitivity to biotin, but most HSN type of vitamins have biotin in them. Because of this I have the tendency to break out when taking anything with it as an incredibly even in the smallest amount. This is something I never was looking to “sacrifice” with other vitamins because they didn’t show much results and I would break out quite a bit, even on my back (I NEVER break out there). My break outs while taking sugar bear were not the worst, but I was getting more than one which I never do. I use a pretty consistent routine that otherwise leaves my skin pretty clear so I feel like I notice everything on my face. Will be repurchasing !"

In [12]:
_input = prompt.format_prompt(title=title_, review=review_)

In [13]:
output = model(_input.to_string())

In [14]:
print(output)


{"Effectiveness": "Positive",
"Price": "Neutral",
"Taste": "Don't Know",
"Side effects": "Negative",
"Quality control": "Neutral",
"Varied results": "Positive",
"Ingredients": "Neutral"}


In [15]:
output

'\n{"Effectiveness": "Positive",\n"Price": "Neutral",\n"Taste": "Don\'t Know",\n"Side effects": "Negative",\n"Quality control": "Neutral",\n"Varied results": "Positive",\n"Ingredients": "Neutral"}'